In [35]:
import pandas as pd
from joblib import dump, load
import unicodedata

# 1. Carregamento dos dados originais
data = {
    'id': [41, 64, 342, 51, 346, 62, 40, 103, 215, 5, 8, 10, 11, 6, 7, 14, 3, 9, 2, 4,
           365, 366, 367, 368, 369, 370, 371, 373, 374, 376, 378, 380, 382, 384, 386,
           388, 390, 394, 399, 401, 403, 405, 407, 409, 411, 412, 413, 415, 417, 419],
    'descricao': ['COMPUTADOR', 'PASTA', 'CANETA', 'PRESENTE', 'MATERIAL DE USO E CONSUMO',
                 'ALIMENTO', 'CABO', 'MIDIA DE ARMAZENAMENTO', 'DRONE', 'SOM AUTOMOTIVO',
                 'GPS', 'TELEVISAO', 'BEBIDA', 'ACESSORIO', 'TINTA', 'MATERIAL DE EMBALAGEM',
                 'MATERIAL DE LIMPEZA', 'ENVELOPE', 'LIVRO', 'CAIXA DE SOM', 'MOVEIS',
                 'SOFAS', 'SERVICOS', 'PARAFUSO', 'SUSPENSAO', 'SERVICO SISTEMA TRANSMISSAO',
                 'SISTEMA DIRECAO', 'SERVICO TERCEIRIZADOS', 'SERVICO INJECAO ELETRONICA',
                 'SERVICO ELETRICO', 'ESTETICA AUTOMOTIVA', 'SERVICO SUSPENSAO', 'SERVICO FREIOS',
                 'SERVICO MOTOR', 'SERVICO SISTEMA DE ARREFECIMENTO', 'SERVICO FUNILARIA E PINTURA',
                 'SERVICO AR CONDICIONADO', 'FREIOS', 'MOTOR', 'SISTEMA ELETRICO',
                 'SISTEMA AR CONDICIONADO', 'SISTEMA ARREFECIMENTO', 'DIRECAO', 'PECAS',
                 'PECAS - BATERIAS', 'PECAS - PNEUS', 'BATERIA', 'PNEUS', 'MATERIAIS DE USO E CONSUMO',
                 'VEICULO']
}

df_original = pd.DataFrame(data)

# 2. Função de normalização corrigida
def normalize_text(text):
    text = str(text)  # Garante que é string
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.lower().strip()
    text = ' '.join(text.split())  # Remove espaços extras
    return text

# 3. Processamento dos dados
df_processed = df_original.copy()
df_processed['descricao'] = df_processed['descricao'].apply(normalize_text)

# Verificação
print("\nAntes da normalização:")
print(df_original['descricao'].head(3))
print("\nDepois da normalização:")
print(df_processed['descricao'].head(3))

# 4. One-Hot Encoding
df_processed = pd.get_dummies(
    df_processed,
    columns=['id', 'descricao'],
    prefix=['id', 'desc'],
    prefix_sep='_'
)

# 5. Salvar os dados
output_data = {
    'original': df_original,
    'processed': df_processed,
    'metadata': {
        'normalization': {
            'steps': ['lowercase', 'remove_accents', 'trim_spaces'],
            'date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        },
        'one_hot_encoding': {
            'columns': ['id', 'descricao'],
            'prefixes': ['id_', 'desc_']
        }
    }
}

dump(output_data, 'padronizado_categorias.z', compress=3)

print("\nArquivo salvo com sucesso!")
print(f"Original: {df_original.shape}")
print(f"Processado: {df_processed.shape}")


Antes da normalização:
0    COMPUTADOR
1         PASTA
2        CANETA
Name: descricao, dtype: object

Depois da normalização:
0    computador
1         pasta
2        caneta
Name: descricao, dtype: object

Arquivo salvo com sucesso!
Original: (50, 2)
Processado: (50, 100)


In [53]:
from joblib import load
import pandas as pd

# Configurar pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

# Carregar os dados
dados = load('padronizado_categorias.z')

# 1. Tabela Processada (Mostrando relação entre valores originais e one-hot)
print("\n" + "═" * 100)
print("TABELA PROCESSADA - RELAÇÃO COMPLETA".center(100))
print("═" * 100)

# Para Criar DataFrame de visualização
df_processados = pd.DataFrame({
    'id ': dados['original']['id'],
    'descrição': dados['original']['descricao'],
    'descrição normalizada': dados['original']['descricao'].str.lower(),
    'coluna id one-hot': dados['processed'].filter(like='id_').idxmax(axis=1),
    'coluna descrição one-hot': dados['processed'].filter(like='desc_').idxmax(axis=1)
})

display(df_processados)

# 2. Metadados Detalhados
print("\n" + "═" * 100)
print("METADADOS DO PROCESSAMENTO".center(100))
print("═" * 100)

# Verificação robusta de metadados
meta = dados.get('metadata', {})
transformacoes = meta.get('transformacoes', {})

print("🔧 NORMALIZAÇÃO APLICADA:")
print(f"- Tipo: {transformacoes.get('normalizacao_texto', {}).get('acao', 'conversão para minúsculas ')}")

print("\n🔩 ONE-HOT ENCODING:")
print(f"- Colunas transformadas: {transformacoes.get('one_hot_encoding', {}).get('colunas', ['id', 'descricao'])}")
print(f"- Prefixos usados: {transformacoes.get('one_hot_encoding', {}).get('prefixos', ['cat_id_', 'cat_desc_'])}")
print(f"- Total de colunas criadas: {len(dados['processed'].columns)}")

print("\n📅 INFORMAÇÕES ADICIONAIS:")
print(f"- Origem dos dados: {meta.get('origem', 'Webgex')}")


════════════════════════════════════════════════════════════════════════════════════════════════════
                                TABELA PROCESSADA - RELAÇÃO COMPLETA                                
════════════════════════════════════════════════════════════════════════════════════════════════════


,id,descrição,descrição normalizada,coluna id one-hot,coluna descrição one-hot
0,41,COMPUTADOR,computador,id_41,desc_computador
1,64,PASTA,pasta,id_64,desc_pasta
2,342,CANETA,caneta,id_342,desc_caneta
3,51,PRESENTE,presente,id_51,desc_presente
4,346,MATERIAL DE USO E CONSUMO,material de uso e consumo,id_346,desc_material de uso e consumo
5,62,ALIMENTO,alimento,id_62,desc_alimento
6,40,CABO,cabo,id_40,desc_cabo
7,103,MIDIA DE ARMAZENAMENTO,midia de armazenamento,id_103,desc_midia de armazenamento
8,215,DRONE,drone,id_215,desc_drone
9,5,SOM AUTOMOTIVO,som automotivo,id_5,desc_som automotivo



════════════════════════════════════════════════════════════════════════════════════════════════════
                                     METADADOS DO PROCESSAMENTO                                     
════════════════════════════════════════════════════════════════════════════════════════════════════
🔧 NORMALIZAÇÃO APLICADA:
- Tipo: conversão para minúsculas 

🔩 ONE-HOT ENCODING:
- Colunas transformadas: ['id', 'descricao']
- Prefixos usados: ['cat_id_', 'cat_desc_']
- Total de colunas criadas: 100

📅 INFORMAÇÕES ADICIONAIS:
- Origem dos dados: Webgex
